In [1]:
from Co2_per_flight import add_emissions_to_flight
from Co2emissionsRoad import add_car_emissions
from SQL_data import get_table_from_db, write_to_emission_db
import pandas as pd

461.646


In [3]:
# flights = flight_id("flightdata.json") # Input fra Avinor API
distance_id = pd.DataFrame(get_table_from_db("distance"), columns=["distance_id", "airport_dep", "airport_arr"])
flights = distance_id.to_dict("records")
flights_with_emissions = add_emissions_to_flight(flights)
flights_with_car_emissions = add_car_emissions(flights_with_emissions)

df = pd.DataFrame(flights_with_car_emissions)

In [4]:
df_copy = df.copy()

In [5]:
df_copy = df_copy.drop(df_copy[(df_copy["airport_dep"] == "OSL") & (df_copy["airport_arr"] == "OSL")].index)

In [6]:
# Adds Co2 emissions for the other direction
df_flip = df_copy.copy()
df_flip["airport_dep"] = df_copy["airport_arr"]
df_flip["airport_arr"] = df_copy["airport_dep"]

In [7]:
df_final = pd.concat([df_copy, df_flip], axis=0, ignore_index=True)

In [8]:
df_final = df_final.sort_values(["airport_dep", "airport_arr"])

In [9]:
df_final = df_final.drop_duplicates(subset=["airport_dep", "airport_arr"], keep="first")


In [10]:
# 
emissions = df_final.to_dict("records")

In [11]:
write_to_emission_db(emissions)